In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error
from tensorflow.keras.callbacks import EarlyStopping,  ModelCheckpoint
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda, MaxPooling1D, Flatten
from keras.models import Sequential
import os
from keras.layers import Dense, Activation, LSTM, Dropout
from sklearn.model_selection import GridSearchCV
import joblib
# import rc
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# how to use .h5 file
# https://www.tensorflow.org/tutorials/keras/save_and_load


import warnings
warnings.filterwarnings('ignore')

c:\Users\hyha1\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


ARIMA 모델 사용

* 경유가격

In [4]:
# 모든 데이터를 합친 데이터(정리된 데이터)
data = pd.read_csv('data/merge_all/merge_drop.csv')

In [5]:
#  거래년월일 컬럼을 2022-10-30 까지만 사용
train = data[data['거래년월일'] <= '2022-09-31']
#y_train = data[data['거래년월일'] <= '2022-10-31']

#  거래년월일 컬럼을 2022-10-30 일부터 사용
test = data[data['거래년월일'] > '2022-09-31']
#y_test = data[data['거래년월일'] > '2022-10-31']

In [6]:
# arima 모델로 예측해보기
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
#------------------------------------------
data1 = train[['거래년월일', '자동차용 경유 가격 (원)']]
data1.index = data1.거래년월일
dt = data1.drop('거래년월일', axis=1)

data2 = test[['거래년월일', '자동차용 경유 가격 (원)']]
data2.index = data2.거래년월일
test_dt = data2.drop('거래년월일', axis=1)

In [7]:

model_arima = ARIMA(dt, order=(1,2,1))
model_arima_fit = model_arima.fit()
print(model_arima_fit.summary())

c:\Users\hyha1\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\hyha1\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\hyha1\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:         자동차용 경유 가격 (원)   No. Observations:                 5741
Model:                 ARIMA(1, 2, 1)   Log Likelihood              -21440.455
Date:                Thu, 29 Dec 2022   AIC                          42886.910
Time:                        11:46:41   BIC                          42906.875
Sample:                             0   HQIC                         42893.859
                               - 5741                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0102      0.015      0.679      0.497      -0.019       0.040
ma.L1         -0.9982      0.001  -1822.688      0.000      -0.999      -0.997
sigma2       102.8207      0.182    563.428      0.0

In [8]:
model_arima_fit.forecast(steps=60)

c:\Users\hyha1\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


5741    1850.860187
5742    1851.527084
5743    1852.194049
5744    1852.861014
5745    1853.527980
5746    1854.194945
5747    1854.861911
5748    1855.528876
5749    1856.195842
5750    1856.862807
5751    1857.529773
5752    1858.196738
5753    1858.863704
5754    1859.530669
5755    1860.197635
5756    1860.864600
5757    1861.531566
5758    1862.198531
5759    1862.865497
5760    1863.532462
5761    1864.199428
5762    1864.866393
5763    1865.533359
5764    1866.200324
5765    1866.867290
5766    1867.534255
5767    1868.201221
5768    1868.868186
5769    1869.535152
5770    1870.202117
5771    1870.869083
5772    1871.536048
5773    1872.203014
5774    1872.869979
5775    1873.536945
5776    1874.203910
5777    1874.870876
5778    1875.537841
5779    1876.204807
5780    1876.871772
5781    1877.538738
5782    1878.205703
5783    1878.872669
5784    1879.539634
5785    1880.206600
5786    1880.873565
5787    1881.540531
5788    1882.207496
5789    1882.874462
5790    1883.541427


In [202]:
test_dt

,자동차용 경유 가격 (원)
거래년월일,
2022-10-04,1838.34
2022-10-05,1838.34
2022-10-06,1838.34
2022-10-07,1838.34
2022-10-11,1838.34
2022-10-12,1838.34
2022-10-13,1838.34
2022-10-14,1838.34
2022-10-17,1838.34


In [ ]:
# 예측값과 실제값 비교
plt.figure(figsize=(10, 5))
plt.plot(test_Y.tolist(), label='y_test')
plt.plot(pred_test, label='y_pred')
plt.legend()
plt.show()